In [2]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [3]:
pip install --upgrade nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
nltk.download('punkt', download_dir='C:\\Users\\devan\\OneDrive\\Desktop\\Assignment_IR')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devan\OneDrive\Desktop\Assignment_IR...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import nltk

In [ ]:
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Question 1

In [7]:
# Import necessary libraries
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import defaultdict
import re

# Download NLTK resources if not already installed
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize stop words, stemmer, and lemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Folder containing the documents
DOCS_FOLDER = "documents"

# Preprocess function to clean, normalize, and tokenize documents
def preprocess(text):
    # Case folding: Convert text to lowercase
    text = text.lower()

    # Tokenization: Split text into words using regex to capture words only
    tokens = re.findall(r'\b\w+\b', text)

    # Stop word removal and stemming/lemmatizing
    processed_tokens = []
    for token in tokens:
        if token not in stop_words:  # Remove stopwords
            stemmed_word = stemmer.stem(token)  # Apply stemming
            lemmatized_word = lemmatizer.lemmatize(stemmed_word)  # Apply lemmatization
            processed_tokens.append(lemmatized_word)
    
    return processed_tokens

# Function to build the inverted index
def build_inverted_index(doc_folder):
    inverted_index = defaultdict(set)  # Dictionary where each word maps to a set of documents containing it
    
    # Loop through the documents in the folder
    for filename in os.listdir(doc_folder):
        if filename.endswith(".txt"):  # Ensure only text files are processed
            doc_id = filename.split(".")[0]  # Extract document ID (e.g., Dell, Zomato, google)
            with open(os.path.join(doc_folder, filename), 'r', encoding='utf-8') as file:
                text = file.read()
                tokens = preprocess(text)  # Preprocess the document text

                # Add tokens to the inverted index
                for token in tokens:
                    inverted_index[token].add(doc_id)
    
    return inverted_index

# Function to handle boolean search queries (AND, OR, NOT)
def boolean_search(query, inverted_index):
    # Preprocess the query in the same way as documents
    query_tokens = preprocess(query)
    
    # Boolean Query processing
    result_docs = set()
    current_operator = "OR"  # Default operator is OR
    temp_result = set()
    
    for token in query_tokens:
        if token in ["and", "or", "not"]:  # Check for boolean operators
            current_operator = token
        else:
            # Get the documents that contain the token from the inverted index
            token_docs = inverted_index.get(token, set())
            
            if current_operator == "OR":
                temp_result = temp_result.union(token_docs)  # Union for OR
            elif current_operator == "AND":
                temp_result = temp_result.intersection(token_docs)  # Intersection for AND
            elif current_operator == "NOT":
                temp_result = temp_result.difference(token_docs)  # Difference for NOT
            
            result_docs = temp_result  # Update the result_docs
    
    return result_docs

# Main function to run the program
def main():
    # Build the inverted index from the document collection
    inverted_index = build_inverted_index(DOCS_FOLDER)

    # Print the inverted index (for verification)
    print("Inverted Index:")
    for term, doc_ids in inverted_index.items():
        print(f"{term}: {sorted(doc_ids)}")

    # Get user input for the boolean search query
    query = input("\nEnter your boolean search query (use AND, OR, NOT): ")

    # Perform the boolean search
    result_docs = boolean_search(query, inverted_index)

    # Print the results
    if result_docs:
        print("\nDocuments matching the query:", sorted(result_docs))
    else:
        print("\nNo documents match the query.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Inverted Index:
adob: ['Adobe']
compani: ['Adobe', 'Amazon', 'Binance', 'Dell', 'HP', 'Lenovo', 'Ola', 'Uber', 'apple', 'blackberry', 'flipkart', 'google', 'huawei', 'levis', 'messenger', 'microsoft', 'motorola', 'nike', 'nokia', 'operating', 'paypal', 'puma', 'reliance', 'samsung', 'sony', 'spotify', 'telegram', 'volkswagen', 'yahoo', 'youtube']
found: ['Adobe', 'Binance', 'Dell', 'HP', 'Lenovo', 'Uber', 'apple', 'blackberry', 'huawei', 'microsoft', 'motorola', 'nokia', 'puma', 'reliance', 'samsung', 'shakespeare', 'spotify', 'steam', 'telegram', 'volkswagen', 'youtube', 'zomato']
1982: ['Adobe', 'HP', 'nokia', 'sony']
john: ['Adobe', 'shakespeare']
warnock: ['Adobe']
charl: ['Adobe', 'shakespeare']
geschk: ['Adobe']
employ: ['Adobe', 'HP', 'Ola', 'Uber', 'apple', 'messenger', 'puma', 'steam', 'telegram']
xerox: ['Adobe']
corpor: ['Adobe', 'Dell', 'HP', 'Lenovo', 'Ola', 'blackberry', 'microsoft', 'motorola', 'nike', 'nokia', 'paypal', 'puma', 'reliance', 'samsung', 'sony', 'spotify', 

Question 2

In [16]:
# Import necessary libraries 
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import defaultdict
import re

# Download NLTK resources if not already installed
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize stop words, stemmer, and lemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Folder containing the documents
DOCS_FOLDER = "documents"

# Preprocess function to clean, normalize, and tokenize documents
def preprocess(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    processed_tokens = []
    for token in tokens:
        if token not in stop_words:  # Remove stopwords
            stemmed_word = stemmer.stem(token)
            lemmatized_word = lemmatizer.lemmatize(stemmed_word)
            processed_tokens.append(lemmatized_word)
    return processed_tokens

# Function to build the inverted index with positional information and biword index
def build_inverted_index(doc_folder):
    inverted_index = defaultdict(set)
    positional_index = defaultdict(lambda: defaultdict(list))  # Stores word positions per document
    biword_index = defaultdict(set)  # Biword index for phrase queries

    # Loop through documents
    for filename in os.listdir(doc_folder):
        if filename.endswith(".txt"):
            doc_id = filename.split(".")[0]  # Extract document ID (e.g., Dell)
            with open(os.path.join(doc_folder, filename), 'r', encoding='utf-8') as file:
                text = file.read()
                tokens = preprocess(text)
                
                # Build inverted index and positional index
                for pos, token in enumerate(tokens):
                    inverted_index[token].add(doc_id)
                    positional_index[token][doc_id].append(pos)
                
                # Build biword index for phrase queries
                for i in range(len(tokens) - 1):
                    biword = tokens[i] + " " + tokens[i+1]
                    biword_index[biword].add(doc_id)

    return inverted_index, positional_index, biword_index

# Function to handle proximity queries
def proximity_search(query, positional_index, k):
    # Extract terms and preprocess them
    terms = preprocess(query)
    if len(terms) != 2:
        raise ValueError("Proximity query must consist of exactly two terms.")
    
    term1, term2 = terms
    result_docs = set()

    # Get documents containing both terms
    if term1 in positional_index and term2 in positional_index:
        docs_with_term1 = positional_index[term1]
        docs_with_term2 = positional_index[term2]
        
        # Check proximity in the documents
        for doc in docs_with_term1.keys() & docs_with_term2.keys():
            positions1 = docs_with_term1[doc]
            positions2 = docs_with_term2[doc]
            for pos1 in positions1:
                for pos2 in positions2:
                    if abs(pos1 - pos2) <= k:
                        result_docs.add(doc)
                        break

    return result_docs

# Function to implement Soundex algorithm for spelling matching
def soundex(name):
    name = name.lower()
    soundex_code = name[0].upper()
    soundex_map = {
        'bfpv': '1', 'cgjkqsxz': '2', 'dt': '3', 'l': '4', 'mn': '5', 'r': '6'
    }

    def encode_char(c):
        for key in soundex_map:
            if c in key:
                return soundex_map[key]
        return ''

    last_code = ''
    for char in name[1:]:
        code = encode_char(char)
        if code != last_code and code:
            soundex_code += code
            last_code = code
        if len(soundex_code) == 4:
            break

    return soundex_code.ljust(4, '0')

# Function to match names phonetically using Soundex
def soundex_search(query, inverted_index):
    query_soundex = soundex(query)
    result_docs = set()

    for term in inverted_index:
        if soundex(term) == query_soundex:
            result_docs.update(inverted_index[term])

    return result_docs

# Function to handle boolean search queries (AND, OR, NOT) with Soundex matching
def boolean_search(query, inverted_index):
    query_parts = re.split(r'\s+(AND|OR|NOT)\s+', query.upper())
    if len(query_parts) % 2 == 0:
        raise ValueError("Query must have a valid format with operators between terms.")

    current_operator = "OR"
    result_docs = set()

    for part in query_parts:
        if part in ["AND", "OR", "NOT"]:
            current_operator = part
        else:
            if part.isalpha() and len(part) > 1:
                term_docs = soundex_search(part, inverted_index)
            else:
                term_docs = inverted_index.get(part.lower(), set())

            if current_operator == "OR":
                result_docs = result_docs.union(term_docs)
            elif current_operator == "AND":
                if not result_docs:  # Initialize result_docs with the first term's docs
                    result_docs = term_docs
                else:
                    result_docs = result_docs.intersection(term_docs)
            elif current_operator == "NOT":
                result_docs = result_docs.difference(term_docs)

    return result_docs

# Main function to run the program
def main():
    # Build the inverted, positional, and biword indices
    inverted_index, positional_index, biword_index = build_inverted_index(DOCS_FOLDER)

    # Print the inverted index (for verification)
    print("Inverted Index:")
    for term, doc_ids in inverted_index.items():
        print(f"{term}: {sorted(doc_ids)}")

    # Print the biword index (for verification)
    print("\nBiword Index:")
    for biword, doc_ids in biword_index.items():
        print(f"{biword}: {sorted(doc_ids)}")

    # Get user input for the query type
    query_type = input("\nEnter query type (boolean, phrase, proximity, soundex): ").strip().lower()
    
    if query_type == "boolean":
        query = input("\nEnter your boolean search query (use AND, OR, NOT): ")
        result_docs = boolean_search(query, inverted_index)
        if result_docs:
            print("\nDocuments matching the query:", sorted(result_docs))
        else:
            print("\nNo documents match the query.")

    elif query_type == "phrase":
        query = input("\nEnter your phrase search query: ")
        query_tokens = preprocess(query)
        if len(query_tokens) < 2:
            print("\nPhrase query must consist of at least two terms.")
        else:
            biword = " ".join(query_tokens[:2])
            result_docs = biword_index.get(biword, set())
            if result_docs:
                print("\nDocuments matching the phrase query:", sorted(result_docs))
            else:
                print("\nNo documents match the phrase query.")

    elif query_type == "proximity":
        query = input("\nEnter your proximity search query (two terms): ")
        k = int(input("\nEnter proximity distance (k): "))
        result_docs = proximity_search(query, positional_index, k)
        if result_docs:
            print("\nDocuments matching the proximity query:", sorted(result_docs))
        else:
            print("\nNo documents match the proximity query.")
    
    elif query_type == "soundex":
        query = input("\nEnter your name query for Soundex matching: ")
        result_docs = soundex_search(query, inverted_index)
        if result_docs:
            print("\nDocuments matching the Soundex query:", sorted(result_docs))
        else:
            print("\nNo documents match the Soundex query.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Inverted Index:
adob: ['Adobe']
compani: ['Adobe', 'Amazon', 'Binance', 'Dell', 'HP', 'Lenovo', 'Ola', 'Uber', 'apple', 'blackberry', 'flipkart', 'google', 'huawei', 'levis', 'messenger', 'microsoft', 'motorola', 'nike', 'nokia', 'operating', 'paypal', 'puma', 'reliance', 'samsung', 'sony', 'spotify', 'telegram', 'volkswagen', 'yahoo', 'youtube']
found: ['Adobe', 'Binance', 'Dell', 'HP', 'Lenovo', 'Uber', 'apple', 'blackberry', 'huawei', 'microsoft', 'motorola', 'nokia', 'puma', 'reliance', 'samsung', 'shakespeare', 'spotify', 'steam', 'telegram', 'volkswagen', 'youtube', 'zomato']
1982: ['Adobe', 'HP', 'nokia', 'sony']
john: ['Adobe', 'shakespeare']
warnock: ['Adobe']
charl: ['Adobe', 'shakespeare']
geschk: ['Adobe']
employ: ['Adobe', 'HP', 'Ola', 'Uber', 'apple', 'messenger', 'puma', 'steam', 'telegram']
xerox: ['Adobe']
corpor: ['Adobe', 'Dell', 'HP', 'Lenovo', 'Ola', 'blackberry', 'microsoft', 'motorola', 'nike', 'nokia', 'paypal', 'puma', 'reliance', 'samsung', 'sony', 'spotify', 